Get the HOGS

In [1]:
from skimage.io import imshow, imread
from skimage.feature import hog
from skimage import exposure
import skimage.transform as transform
import skimage.io
import skimage.exposure
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import os
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np

In [2]:
def read_data(folder_path):
    X = []
    y = []

    for class_label in range(43):
        class_folder = os.path.join(folder_path, str(class_label))
        
        for file_name in os.listdir(class_folder):
            if file_name.endswith(".ppm"):
                file_path = os.path.join(class_folder, file_name)
                
                # Code snippet for hogz
                img = skimage.io.imread(file_path, plugin='matplotlib')
                img_gray = skimage.color.rgb2gray(img)
                reshaped_img = transform.resize(img_gray, (40, 40))
                normalized_image = skimage.exposure.rescale_intensity(reshaped_img, in_range='image', out_range=(0, 1))
                hog_features, hog_image = hog(normalized_image, orientations=8, pixels_per_cell=(6, 6), cells_per_block=(2, 2), visualize=True)
                hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 5))
                hog_features_array = hog_features.reshape(-1)
                X.append(hog_features_array)
                y.append(class_label)

    return np.array(X), np.array(y)

folder_path_train = "/home/jovyan/DATA/dataset_folder_train_images/GTSRB/Final_Training/Images"

X,y = read_data(folder_path_train)

In [3]:
#random Forest
X_train_forest, X_test_forest, y_train_forest, y_test_forest = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled_forest = scaler.fit_transform(X_train_forest)
X_test_scaled_forest = scaler.transform(X_test_forest)

# Create and train the Random Forest model
random_forest = RandomForestClassifier()
random_forest.fit(X_train_scaled_forest, y_train_forest)

# Make predictions on the test set
y_pred_forest = random_forest.predict(X_test_scaled_forest)

# Evaluate the model
accuracy_forest = accuracy_score(y_test_forest, y_pred_forest)
print("Accuracy for Random Forest:", accuracy_forest)
print()

# Calculating precision, recall, and F1-score for each class
precision_forest = precision_score(y_test_forest, y_pred_forest, average=None)
recall_forest = recall_score(y_test_forest, y_pred_forest, average=None)
f1_forest = f1_score(y_test_forest, y_pred_forest, average=None)

# Printing the evaluation metrics for each class
for class_label, prec, rec, f1_score in zip(np.unique(y), precision_forest, recall_forest, f1_forest):
    print(f"Class: {class_label}")
    print(f"Precision: {prec}")
    print(f"Recall: {rec}")
    print(f"F1-Score: {f1_score}")
    print()

Accuracy for Random Forest: 0.9468247895944912
Class: 0
Precision: 1.0
Recall: 0.6052631578947368
F1-Score: 0.7540983606557378

Class: 1
Precision: 0.8937875751503006
Recall: 0.8991935483870968
F1-Score: 0.8964824120603015

Class: 2
Precision: 0.8357289527720739
Recall: 0.9044444444444445
F1-Score: 0.8687299893276414

Class: 3
Precision: 0.9411764705882353
Recall: 0.9142857142857143
F1-Score: 0.9275362318840579

Class: 4
Precision: 0.9178082191780822
Recall: 0.9617224880382775
F1-Score: 0.9392523364485982

Class: 5
Precision: 0.8149171270718232
Recall: 0.8104395604395604
F1-Score: 0.8126721763085398

Class: 6
Precision: 0.9833333333333333
Recall: 1.0
F1-Score: 0.9915966386554621

Class: 7
Precision: 0.899641577060932
Recall: 0.9028776978417267
F1-Score: 0.9012567324955117

Class: 8
Precision: 0.9261992619926199
Recall: 0.8338870431893688
F1-Score: 0.8776223776223776

Class: 9
Precision: 0.9777777777777777
Recall: 0.9850746268656716
F1-Score: 0.9814126394052044

Class: 10
Precision: 0.9

In [3]:
#SVM
X_train_svm, X_test_svm, y_train_svm, y_test_svm = train_test_split(X, y, test_size=0.2, random_state=42)

clf= make_pipeline(StandardScaler(),SVC(kernel="rbf", gamma= "auto"))
clf.fit(X_train_svm,y_train_svm)
y_pred_svm = clf.predict(X_test_svm)
accuracy_svm = accuracy_score(y_test_svm, y_pred_svm)
print("Accuracy for a SVM:", accuracy_svm)
print()

# Calculating precision, recall, and F1-score for each class
precision_svm = precision_score(y_test_svm, y_pred_svm, average=None)
recall_svm = recall_score(y_test_svm, y_pred_svm, average=None)
f1_svm = f1_score(y_test_svm, y_pred_svm, average=None)

# Printing the evaluation metrics for each class
for class_label, prec, rec, f1_score in zip(np.unique(y), precision_svm, recall_svm, f1_svm):
    print(f"Class: {class_label}")
    print(f"Precision: {prec}")
    print(f"Recall: {rec}")
    print(f"F1-Score: {f1_score}")
    print()

Accuracy for a SVM: 0.9802346340219332

Class: 0
Precision: 1.0
Recall: 0.9210526315789473
F1-Score: 0.958904109589041

Class: 1
Precision: 0.945010183299389
Recall: 0.9354838709677419
F1-Score: 0.9402228976697062

Class: 2
Precision: 0.9166666666666666
Recall: 0.9533333333333334
F1-Score: 0.934640522875817

Class: 3
Precision: 0.9818181818181818
Recall: 0.9642857142857143
F1-Score: 0.972972972972973

Class: 4
Precision: 0.9809976247030879
Recall: 0.9880382775119617
F1-Score: 0.9845053635280095

Class: 5
Precision: 0.9078212290502793
Recall: 0.8928571428571429
F1-Score: 0.9002770083102493

Class: 6
Precision: 0.9833333333333333
Recall: 1.0
F1-Score: 0.9915966386554621

Class: 7
Precision: 0.9853479853479854
Recall: 0.9676258992805755
F1-Score: 0.9764065335753176

Class: 8
Precision: 0.9634551495016611
Recall: 0.9634551495016611
F1-Score: 0.9634551495016611

Class: 9
Precision: 1.0
Recall: 0.996268656716418
F1-Score: 0.9981308411214954

Class: 10
Precision: 0.9973045822102425
Recall: 1.